In [ ]:
#Imports
import numpy as np
import pandas as pd 
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input,Conv2D, Flatten, Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import random
import cv2

### Data Loading

In [ ]:
#List of image category
list_dir = []
base_dir = "../input/food41/images"
for i in os.listdir(base_dir):
    i = os.path.join(base_dir, i)
    list_dir.append(i)

In [ ]:
image_path = []
# Getting all the image paths
for l in list_dir[:2]:
    for i in os.listdir(l):
        path = os.path.join(l, i)
        image_path.append(path)

In [ ]:
# Randomly Shuffling all the image path
random.shuffle(image_path)

In [ ]:
#Loding and checking the size of a image
img = cv2.imread(image_path[0])
plt.imshow(img)
print(img.shape)

In [ ]:
# Loading all the images
y = []
for p in image_path:
    img= Image.open(p)
    img= img.resize((256, 256)) #Resizing the images
    img= np.array(img) # Converting images to array
    img = img / 255.0 # Normalizing images for faster conversion
    y.append(img)

In [ ]:
# Converting image to Black and White
x = []
for p in image_path:
    img= Image.open(p)
    img= img.convert("L") # Converting Image to greayscale
    img= img.resize((256, 256))
    img= np.array(img)
    img = img / 255.0
    x.append(img)

In [ ]:
# Converting the lists to array
x = np.array(x)
x= x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
y = np.array(y)

In [ ]:
# Seperating train and test data
seperation = 1500
x_train = x[:seperation]
y_train = y[:seperation]
x_test = x[seperation:]
y_test = y[seperation:]

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### Model Building

In [ ]:
# Hyperperameters
cnn_filters = [32, 32, 64, 64,128]
input_shape = (256, 256, 1)
batch_size = 64
kernel_size = 3
latent_dim =  512

In [96]:
def build_encoder(input_shape, cnn_filters, kernel_size, latent_dim):
    '''
    Building the  encoder part of the model. The Encoder has 5 Conv2D layes having filter size of 32, 32, 64, 64, 128, one Flatten layer and one Dense Layer.
    The task of the encoder is to encode the image into a  latent dimension space. From which the decoder will try to regenerate the output image.
    '''
    inputs = Input(input_shape, name='encoder_input')
    x = inputs
    for filters in cnn_filters:
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=2,
                   activation='relu',
                   padding='same')(x)
        
    shape = K.int_shape(x)
    x= Flatten()(x)
    latent_space = Dense(latent_dim, name='latent_vector')(x)
    encoder= Model(inputs, latent_space, name='encoder')
    encoder.summary()
    return encoder, shape, inputs

In [97]:
def build_decoder(cnn_filters, kernel_size, latent_dim, shape, channels=3):
    '''
    Building the  decoder part of the model. The Decoder has 6 Conv2DTranspose layes having filter size of  128, 64, 64, 32, 32, 3 , 
    One Dense Layer and a Reshape Layer. The task of the decoder is to recreate  the image from  a  latent 
    dimension space and will try to regenerate the desired output image.
    '''
    latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
    x = Dense(shape[1]*shape[2]*shape[3])(latent_inputs)
    x = Reshape((shape[1], shape[2], shape[3]))(x)
    for filters in cnn_filters[::-1]:
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=2,
                            activation='relu',padding='same')(x)
        outputs = Conv2DTranspose(filters=channels,
                                  kernel_size=kernel_size, activation='sigmoid',
                                  padding='same',name='decoder_output')(x)
        
    decoder = Model(latent_inputs, outputs, name='decoder')
    decoder.summary()
    return decoder

In [98]:
def build_autoencoder(encoder, decoder, inputs):
    '''
    autoencoder = encoder + decoder
    '''
    autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')
    autoencoder.summary()
    return autoencoder

In [99]:
encoder, shape, inputs = build_encoder(input_shape, cnn_filters, kernel_size, latent_dim)

In [100]:
decoder = build_decoder(cnn_filters, kernel_size, latent_dim, shape)

In [101]:
autoencoder = build_autoencoder(encoder, decoder, inputs)

In [102]:
plot_model(autoencoder, "auto.png")

In [103]:
# Learning Rate Scheduler
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=3,
                              verbose=1)

# Stopping early if the model is not improving
es_cb= EarlyStopping(monitor='val_loss', patience=5, verbose=1,)

In [104]:
autoencoder.compile(loss='mse', optimizer='adam')

In [105]:
# Model Training
history = autoencoder.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size , epochs= 50, callbacks=[lr_reducer, es_cb])

In [144]:
# Plotting the graph
plt.plot(history.history["val_loss"], label= "val_loss")
plt.plot(history.history["loss"], label= "val_loss")
plt.legend()
plt.savefig("loss.pdf")

In [134]:
#Checking the output
trial_x = x_test[140]
trial_y = y_test[140]

In [135]:
# Reshaping the data to fit in the model to make prediction
test_image = trial_x.reshape((1, 256, 256, 1))

In [136]:
# Making the prediction
pred = autoencoder.predict(test_image)

In [137]:
#Actual Image
plt.imshow(trial_y, cmap="gray")
plt.savefig("true.pdf")

In [138]:
# Merging the predicted and input Image
pred = pred+ test_image

In [139]:
# Predicted Result
plt.imshow(pred[0])
plt.savefig("pred.pdf")

In [140]:
# Input Image
plt.imshow(trial_x, cmap="gray")
plt.savefig("input.pdf")

In [142]:
# Saving the model
autoencoder.save("colorization.h5")